In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix
GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
LENGTH = 4.5  # [m]
WIDTH = 2.0  # [m]
BACKTOWHEEL = 1.0  # [m]
WHEEL_LEN = 0.3  # [m]
WHEEL_WIDTH = 0.2  # [m]
TREAD = 0.7  # [m]
WB = 110#2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True



        

def pi_2_pi(angle):
    while(angle > math.pi):
        angle = angle - 2.0 * math.pi

    while(angle < -math.pi):
        angle = angle + 2.0 * math.pi

    return angle





def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", truckcolor="-k"):  # pragma: no cover

    outline = np.array([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                        [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.array([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                         [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.array([[math.cos(yaw), math.sin(yaw)],
                     [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.array([[math.cos(steer), math.sin(steer)],
                     [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T.dot(Rot2)).T
    fl_wheel = (fl_wheel.T.dot(Rot2)).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T.dot(Rot1)).T
    fl_wheel = (fl_wheel.T.dot(Rot1)).T

    outline = (outline.T.dot(Rot1)).T
    rr_wheel = (rr_wheel.T.dot(Rot1)).T
    rl_wheel = (rl_wheel.T.dot(Rot1)).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    plt.plot(np.array(outline[0, :]).flatten(),
             np.array(outline[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fr_wheel[0, :]).flatten(),
             np.array(fr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rr_wheel[0, :]).flatten(),
             np.array(rr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fl_wheel[0, :]).flatten(),
             np.array(fl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rl_wheel[0, :]).flatten(),
             np.array(rl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(x, y, "*")




def get_nparray_from_matrix(x):
    return np.array(x).flatten()


def get_closest_point(state, ccx, ccy, cyaw, pind):
    print(ccx,ccy)
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in zip(ccx,ccy)]
    print('printing d',d,len(d),type(d))
    mind = min(d)
    ind = d.index(mind) + pind
    print('ds',ind,mind)
    return ind, mind




def ccalc_ref_trajectory(state, cx, cy, cyaw, ck, sp, dl, pind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    ind, _ = get_closest_point(state, cx, cy, cyaw, pind)

    if pind >= ind:
        ind = pind

    xref[0, 0] = cx[ind]
    xref[1, 0] = cy[ind]
    xref[2, 0] = sp[ind]
    xref[3, 0] = cyaw[ind]
    dref[0, 0] = 0.0  # steer operational point should be 0

    S = 0.0
    vel=state.v
    for i in range(T + 1):
        S += abs(vel) * DT
        vel+=acc*DT
        dind = int(round(travel / dl))

        if (ind + dind) < ncourse:
            xref[0, i] = cx[ind + dind]
            xref[1, i] = cy[ind + dind]
            xref[2, i] = sp[ind + dind]
            xref[3, i] = cyaw[ind + dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = cx[ncourse - 1]
            xref[1, i] = cy[ncourse - 1]
            xref[2, i] = sp[ncourse - 1]
            xref[3, i] = cyaw[ncourse - 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def calc_ref_trajectory(state, ccx, ccy, ccyaw, ck, ssp, dl, ind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    #ind, _ = get_closest_point(state, ccx, ccy, ccyaw, pind)
    #if pind >= ind:
    #    ind = pind

    xref[0, 0] = ccx[0]
    xref[1, 0] = ccy[0]
    xref[2, 0] = ssp[0]
    xref[3, 0] = ccyaw[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(ccx)-1:
            xref[0, i] = ccx[dind]
            xref[1, i] = ccy[dind]
            xref[2, i] = ssp[dind]
            xref[3, i] = ccyaw[dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = ccx[-1]
            xref[1, i] = ccy[-1]
            xref[2, i] = ssp[- 1]
            xref[3, i] = ccyaw[- 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False




def calc_speed_profile(cx, cy, cyaw, target_speed):

    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile


def smooth_yaw(yaw):

    for i in range(len(yaw) - 1):
        dyaw = yaw[i + 1] - yaw[i]

        while dyaw >= math.pi / 2.0:
            yaw[i + 1] -= math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

        while dyaw <= -math.pi / 2.0:
            yaw[i + 1] += math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

    return yaw


def get_straight_course(dl):
    ax = [0.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0]
    ay = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course2(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course3(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    cyaw = [i - math.pi for i in cyaw]

    return cx, cy, cyaw, ck


def get_forward_course(dl):
    ax = [0.0, 60.0, 125.0, 50.0, 75.0, 30.0, -10.0]
    ay = [0.0, 0.0, 50.0, 65.0, 30.0, 50.0, -20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_switch_back_course(dl):
    ax = [0.0, 30.0, 6.0, 20.0, 35.0]
    ay = [0.0, 0.0, 20.0, 35.0, 20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    ax = [35.0, 10.0, 0.0, 0.0]
    ay = [20.0, 30.0, 5.0, 0.0]
    cx2, cy2, cyaw2, ck2, s2 = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    cyaw2 = [i - math.pi for i in cyaw2]
    cx.extend(cx2)
    cy.extend(cy2)
    cyaw.extend(cyaw2)
    ck.extend(ck2)

    return cx, cy, cyaw, ck


def main():
    

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)




def main2():
    print(__file__ + " start!!")

    dl = 1.0  # course tick
    cx, cy, cyaw, ck = get_straight_course3(dl)

    sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)

    initial_state = State(x=cx[0], y=cy[0], yaw=0.0, v=0.0)

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)

    if show_animation:  # pragma: no cover
        plt.close("all")
        plt.subplots()
        plt.plot(cx, cy, "-r", label="spline")
        plt.plot(x, y, "-g", label="tracking")
        plt.grid(True)
        plt.axis("equal")
        plt.xlabel("x[m]")
        plt.ylabel("y[m]")
        plt.legend()

        plt.subplots()
        plt.plot(t, v, "-r", label="speed")
        plt.grid(True)
        plt.xlabel("Time [s]")
        plt.ylabel("Speed [kmh]")

        plt.show()

In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od,x0):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref,x0)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")
    
    return oa, od, ox, oy, oyaw, ov,xbar

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref,x0):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = get_nparray_from_matrix(x.value[0, :])
        oy = get_nparray_from_matrix(x.value[1, :])
        ov = get_nparray_from_matrix(x.value[2, :])
        oyaw = get_nparray_from_matrix(x.value[3, :])
        oa = get_nparray_from_matrix(u.value[0, :])
        odelta = get_nparray_from_matrix(u.value[1, :])

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta):

        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = self.state.x + self.state.v * math.cos(self.state.yaw) * DT
        next_y = self.state.y + self.state.v * math.sin(self.state.yaw) * DT
        next_yaw = self.state.yaw + self.state.v / WB * math.tan(delta) * DT
        next_v = self.state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw

        return xbar


In [4]:
def gen_points(cx,cy):
    dx=20#2
    dy=10#1
    pts=[[cx-dx,cy+dy],
    [cx+dx,cy+dy],
    [cx+dx,cy-dy],
    [cx-dx,cy-dy],
    [cx-dx,cy+dy]]
    pts=np.array(pts,dtype=np.float64)
    return pts
class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points

In [5]:
%matplotlib tk
import gym
from matplotlib import pyplot as plt
import cv2
import numpy as np
from car_env import CarRacing
import time
import cubic_spline_planner
def rad_to_deg():
    pass
def accl_to_gas(accl,vel):
    SIZE = 0.02
    ENGINE_POWER= 100000000*SIZE*SIZE
    WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
    WHEEL_R  = 27 
    wheel_rad=WHEEL_R*SIZE
    mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
    #mass=.25
    power=4*mass*accl*(vel+accl/2)
    gas=power/ENGINE_POWER
    return gas

def env_step(env,action,show_plot=False):
    
    frame=np.zeros((100,200,3),dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    #action[0]=action[0]-np.deg2rad(90)
    action[0]*=-1
    nst,rw,ter,info=envc.step(action) # take a random action
    frame[0:96,0:96,:]=np.array(nst)
    
    pos=envc.car.hull.position
    vel=np.sqrt(envc.car.hull.linearVelocity[0]**2+envc.car.hull.linearVelocity[1]**2)
    ang=envc.car.hull.angle
    #ang*=-1
    ang=ang+np.deg2rad(90)#-ang
    if show_plot==True:
        plt.cla()
        img = cv2.putText(np.array(frame), 'vel %f'%vel, (100,10), font,.3, (255,255,0),1, cv2.LINE_AA)
        img = cv2.putText(np.array(img), 'ang %f'%ang, (100,20), font,.3, (255,0,255),1, cv2.LINE_AA)
        image = cv2.putText(np.array(img), 'x,y %f,%f'%(pos[0],pos[1]), (100,30), font,.2, (255,0,255),1, cv2.LINE_AA)
       
        plt.imshow(image)
        plt.show()
        plt.pause(0.0001)
        
    return [pos[0],pos[1],ang,vel],rw,ter,info
#envc.close()


envc=CarRacing()
def env_reset(envc):
    import pickle
    envc.reset()
    save=False
    if save==True:
        f=open('path-1','wb')
        pickle.dump(envc.track,f)
        f.close()
    f=open('path-1','rb')
    track=pickle.load(f)
    f.close()
    envc.track=track
    
    points=[[x,y] for a,b,x,y in envc.track]
    points=np.array(points)
    plt.plot(points[:,0],points[:,1])
    plt.show()
    return [env_step(envc,[0,0,0]),points]
    
#st,points=env_reset(envc)
'''
for i in range(100):
    st,rw,ter,info=env_step(envc,[0,1,0])
    time.sleep(0.1)
    #print(st)
    '''
#envc.close()



'\nfor i in range(100):\n    st,rw,ter,info=env_step(envc,[0,1,0])\n    time.sleep(0.1)\n    #print(st)\n    '

In [6]:
envc.close()

In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
envc=CarRacing()
[init_state,traj_points]=env_reset(envc)
ix,iy,iyaw,ivel=init_state[0]
tx,ty=traj_points[:50,0],traj_points[:50,1]

cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
#yaw0=np.rad2deg(90)
#cyaw=[yaw-yaw0 for yaw in cyaw]
#dcx=np.gradient(cx)
#dcy=np.gradient(cy)
#cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]
sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)


Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
print(state)

Track generation: 1204..1509 -> 305-tiles track


In [8]:
def get_closest_point(state, ccx, ccy, cyaw, pind):
    #print(ccx,ccy,state.x,state.y)
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in zip(ccx,ccy)]
    #print('printing d',d,len(d),type(d))
    mind = min(d)
    ind = d.index(mind) + pind
    #print('ds',ind,mind)
    return ind, mind

def calc_ref_trajectory(state, ccx, ccy, ccyaw, ck, ssp, dl, ind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    #ind, _ = get_closest_point(state, ccx, ccy, ccyaw, pind)
    #if pind >= ind:
    #    ind = pind

    xref[0, 0] = ccx[0]
    xref[1, 0] = ccy[0]
    xref[2, 0] = ssp[0]
    xref[3, 0] = ccyaw[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(ccx)-1:
            xref[0, i] = ccx[dind]
            xref[1, i] = ccy[dind]
            xref[2, i] = ssp[dind]
            xref[3, i] = ccyaw[dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = ccx[-1]
            xref[1, i] = ccy[-1]
            xref[2, i] = ssp[- 1]
            xref[3, i] = ccyaw[- 1]
            dref[0, i] = 0.0

    return xref, ind, dref

In [9]:
# initial yaw compensation
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
Motion_Status.set_state(state)
goal = [cx[-1], cy[-1]]

In [10]:
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]
def get_raw_sensor_value():
    pass
_cx=cx[:10]
_cy=cy[:10]
_cyaw=cyaw[:10]
_sp=sp[:10]
#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
index=0
cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        print("x,y,yaw,v\n",state.x,state.y,state.yaw,state.v)
        state_str=str(state.x)+','+str(state.y)+','+str(state.v)+','+str(np.rad2deg(state.yaw))+'\n'
        #state_str=str(state.yaw)+'\n'
        
        index, _ = get_closest_point(state, _cx, _cy, cyaw, index)
        #if pind >= index:
        #    index = pind
        _cx=cx[index:index+15]
        _cy=cy[index:index+15]
        _cyaw=cyaw[index:index+15]
        _sp=sp[index:index+15]
        
        pind=index
        xref,_, dref = calc_ref_trajectory(state, _cx, _cy,_cyaw, ck, _sp, dl, index)
        
        
        print('xref...............\n',xref,'\n')
        x0 = [state.x, state.y, state.v, state.yaw]  # current state

        oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(
            xref, Motion_Status, dref, oa, odelta,x0)
        xref_str=str(xref[0][0])+','+str(xref[1][0])+','+str(xref[2][0])+','+str(np.rad2deg(xref[3][0]))+','+str(index)+'\n'
        xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        if odelta is not None:
            di, ai = odelta[0], oa[0]
        accl=ai
        gas=accl_to_gas(accl,state.v)
        steer=di
        print('accl,gas,steer\n',accl,gas,steer)
        act_str=str(accl)+','+str(gas)+','+str(np.rad2deg(steer))+'\n'
        #act_str=str(steer)+'\n'
        nst,rw,ter,info=env_step(envc,[steer,gas,0])
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        #next_state = Motion_Status.get_next_state(ai, di)
        Motion_Status.set_state(next_state)
        
        time = time + DT

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        if check_goal(state, goal, index, len(cx)):
            print("Goal")
            break

        if show_animation:  # pragma: no cover
            plt.cla()
            plt.text(50,50,'state : '+state_str,fontsize=12)
            plt.text(50,40,'xref : '+xref_str,fontsize=12)
            plt.text(50,30,'xbar : '+xbar_str,fontsize=12)
            plt.text(50,20,'act : '+act_str,fontsize=12)
            
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(x, y, "ob", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(cx[index], cy[index], "xg", label="target")
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            points=gen_points(state.x,state.y)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=move_points(points,0,0,ang_yaw)
            r_points=rp.apply()
            points=np.array(points,dtype=np.float64)
            r_points=np.array(r_points,dtype=np.float64)
            plt.plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            plt.title("Time[s]:" + str(round(time, 2))
                      + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
                      + '\nyaw'+str(state.yaw)
                      + '\nxref_yaw'+str(xref[3][0])
                      + '\nsteer'+str(steer))
            plt.pause(0.0001)

if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


x,y,yaw,v
 219.59217834472656 -52.24860763549805 1.4718374768879752 8.108147214479303e-07
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.77777778   2.77777778
    2.77777778]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 1.0000000002349432 0.00029629677691831986 -0.00045834041572183576
215.59217834472656 -50.24860763549805
x,y,yaw,v
 219.59219360351562 -52.24846267700195 1.471848108866487 0.007187189758905774
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.77777778   2.77777778
    2.77777778]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.469

accl,gas,steer
 1.0000000002519853 0.0003497851912438952 -0.0011776033690186804
215.593505859375 -50.23525619506836
x,y,yaw,v
 219.5937042236328 -52.23332595825195 1.471843727925096 0.09708341413521838
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.77777778   2.77777778
    2.77777778]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 1.00000000002411 0.00035382720839210015 -0.0012442384748393063
215.5937042236328 -50.23332595825195
x,y,yaw,v
 219.5939178466797 -52.231258392333984 1.4718431393292288 0.10396584088029255
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.777777

accl,gas,steer
 1.000000000025905 0.00041431172581160754 -0.002447958235360325
215.59788513183594 -50.19132995605469
x,y,yaw,v
 219.59829711914062 -52.18721389770508 1.4718257869270186 0.20692034340877566
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.77777778   2.77777778
    2.77777778]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 1.0000000000271707 0.000418915759076485 -0.00255532206130827
215.59829711914062 -50.18721389770508
x,y,yaw,v
 219.59872436523438 -52.18294143676758 1.4718234623458724 0.21475458847442452
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.7777

accl,gas,steer
 0.9999999999965162 0.00048745989421496454 -0.00441230740514671
215.60581970214844 -50.11260223388672
x,y,yaw,v
 219.6064910888672 -52.10601043701172 1.4717586050417761 0.3313496832049244
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.77777778   2.77777778
    2.77777778]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0.9999999999886714 0.000492651664112499 -0.004572364809317758
215.6064910888672 -50.10601043701172
x,y,yaw,v
 219.607177734375 -52.0992431640625 1.4717509979989867 0.34017803883012077
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.73870516]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -52.03212331]
 [  2.77777778   2.77777778   2.77777778   2.77777778 

accl,gas,steer
 1.0000000000188036 0.0005695521308227968 -0.007323041317872575
215.61785888671875 -49.994651794433594
x,y,yaw,v
 219.6188201904297 -51.98528289794922 1.4715788150813918 0.47090028404872586
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.84009789]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -51.03727683]
 [  2.77777778   2.77777778   2.77777778   2.77777778   2.77777778
    2.77777778]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 1.0000000000080884 0.0005753483164803693 -0.007550044572569552
215.6188201904297 -49.98528289794922
x,y,yaw,v
 219.6197967529297 -51.9757194519043 1.4715604866531233 0.48074994739356947
xref...............
 [[219.73870516 219.73870516 219.73870516 219.73870516 219.73870516
  219.84009789]
 [-52.03212331 -52.03212331 -52.03212331 -52.03212331 -52.03212331
  -51.03727683]
 [  2.77777778   2.77777778   2.77777778   2.77777

...

x,y,yaw,v
 115.5438232421875 59.350711822509766 4.265066222348485 3.7952833849765657
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.9999998164076901 -0.0019527602203164537 0.7853961295413411
111.5438232421875 61.350711822509766
x,y,yaw,v
 115.52737426757812 59.27645492553711 4.275261239209447 3.7935818981979166
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.

x,y,yaw,v
 115.37747955322266 58.226810455322266 4.417512015500341 3.769850676419307
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.9999986390003286 -0.0019376870557940862 0.7853849709923553
111.37747955322266 60.226810455322266
x,y,yaw,v
 115.37252807617188 58.15142822265625 4.427638606229101 3.768162155706018
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.

x,y,yaw,v
 115.38318634033203 57.09827423095703 4.56893523136643 3.744604431260398
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.9999997444332939 -0.00192272813619606 0.7853960546823416
111.38318634033203 59.09827423095703
x,y,yaw,v
 115.38961029052734 57.02350997924805 4.578993872800146 3.7429277040996594
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.9999

x,y,yaw,v
 115.55735778808594 55.990882873535156 4.719344214597021 3.7195410476185455
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.9999997090920072 -0.0019078757075450521 0.7853961018029673
111.55735778808594 57.990882873535156
x,y,yaw,v
 115.57479858398438 55.91841506958008 4.72933562199143 3.717877220917413
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.

x,y,yaw,v
 115.89275360107422 54.92906951904297 4.868746117749486 3.6946610124641763
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.9999989269377069 -0.001893130738325615 0.7853915934824499
111.89275360107422 56.92906951904297
x,y,yaw,v
 115.92059326171875 54.86046600341797 4.878670529523168 3.693008014245725
xref...............
 [[131.69496636 131.69496636 131.69496636 131.69496636 131.69496636
  131.69496636]
 [ 61.22572345  61.22572345  61.22572345  61.22572345  61.22572345
   61.22572345]
 [  0.           0.           0.           0.           0.
    0.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 -0.99

In [11]:
len(cx)

172

In [12]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
i_yaw=cyaw[0]
cyaw=[yaw-i_yaw for yaw in cyaw]
cyaw

[0.0,
 1.1365353103087728e-11,
 4.5461634456955835e-11,
 1.0228884406160432e-10,
 1.6479884124009914e-10,
 1.3070255988623103e-10,
 -1.7048140676934054e-11,
 -2.7845348249400104e-10,
 -3.8074232655560536e-10,
 -5.1144644075407086e-11,
 7.103400090358036e-10,
 1.6479890785348061e-09,
 1.2274679050960913e-09,
 -8.069465096127715e-10,
 -4.455253721502572e-09,
 -5.898664889869565e-09,
 -1.3183913960546079e-09,
 9.285566981986904e-09,
 2.235014373574984e-08,
 1.649694048033723e-08,
 -1.1837108848666844e-08,
 -6.265200425126238e-08,
 -8.27574726525171e-08,
 -1.89632416436325e-08,
 1.2873069166197126e-07,
 3.106971246946699e-07,
 2.2917279784984146e-07,
 -1.65469482116265e-07,
 -8.732296441493759e-07,
 -1.153262739972405e-06,
 -2.6472389191845025e-07,
 1.7923874966463416e-06,
 4.326854357694643e-06,
 3.1913662261739972e-06,
 -2.305292075233112e-06,
 -1.2163106779983224e-05,
 -1.6063454876258376e-05,
 -3.6877269422586068e-06,
 2.4964192695930976e-05,
 6.0265029441453066e-05,
 4.444957289728712

In [13]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [14]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [15]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [16]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
yaw0=np.deg2rad(90)
cyaw=[yaw-yaw0 for yaw in cyaw]

In [17]:
import numpy as np
np.rad2deg(2.64)

151.26085791453733

In [ ]:
cyaw